In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
tqdm.pandas()
from IPython.display import display


/Users/sebastiancoombs/miniconda3/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Begining of the project


In [2]:
reg_season=pd.read_csv('/Users/sebastiancoombs/Downloads/march-machine-learning-mania-2023/WRegularSeasonDetailedResults.csv')
reg_season

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
0,2010,11,3103,63,3237,49,H,0,23,54,...,13,6,10,11,27,11,23,7,6,19
1,2010,11,3104,73,3399,68,N,0,26,62,...,21,14,27,14,26,7,20,4,2,27
2,2010,11,3110,71,3224,59,A,0,29,62,...,14,19,23,17,23,8,15,6,0,15
3,2010,11,3111,63,3267,58,A,0,27,52,...,26,16,25,22,22,15,11,14,5,14
4,2010,11,3119,74,3447,70,H,1,30,74,...,17,11,21,21,32,12,14,4,2,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70778,2023,132,3179,89,3125,71,N,0,31,53,...,26,2,6,11,17,14,8,6,2,16
70779,2023,132,3221,66,3131,61,A,0,23,44,...,17,11,16,6,13,7,6,11,3,20
70780,2023,132,3235,61,3400,51,N,0,20,59,...,14,6,14,19,26,13,14,5,12,19
70781,2023,132,3284,80,3406,55,A,0,26,51,...,14,5,7,10,16,8,15,6,2,20


In [3]:
conf = pd.read_csv('/Users/sebastiancoombs/Downloads/march-machine-learning-mania-2023/WTeamConferences.csv')
conf['TeamID']=conf.Season.astype(str)+'_'+conf.TeamID.astype(str)
conf

,Season,TeamID,ConfAbbrev
0,1998,1998_3102,wac
1,1998,1998_3103,mac
2,1998,1998_3104,sec
3,1998,1998_3106,swac
4,1998,1998_3108,swac
...,...,...,...
8763,2023,2023_3473,ovc
8764,2023,2023_3474,a_sun
8765,2023,2023_3475,ovc
8766,2023,2023_3476,nec


In [4]:
reg_season[reg_season['WTeamID']==1246].Season.unique()

array([], dtype=int64)

In [5]:

reg_season['WTeamID']=reg_season.Season.astype(str)+'_'+reg_season.WTeamID.astype(str)
reg_season['LTeamID']=reg_season.Season.astype(str)+'_'+reg_season.LTeamID.astype(str)
reg_season.head(3)
reg_season.Season.unique()

array([2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020,
       2021, 2022, 2023])

In [6]:
w_games=reg_season.filter(regex='^W',).copy()
w_games=pd.concat([reg_season['Season'],w_games],axis=1)
w_games.columns=[w.replace('W','',1) for w in w_games.columns]

l_games=reg_season.filter(regex='^L',).copy()
l_games=pd.concat([reg_season['Season'],l_games],axis=1)
l_games.columns=[l.replace('L','',1) for l in l_games.columns]


In [7]:
w_games

,Season,TeamID,Score,Loc,FGM,FGA,FGM3,FGA3,FTM,FTA,OR,DR,Ast,TO,Stl,Blk,PF
0,2010,2010_3103,63,H,23,54,5,9,12,19,10,26,14,18,7,0,15
1,2010,2010_3104,73,N,26,62,5,12,16,28,16,31,15,20,5,2,25
2,2010,2010_3110,71,A,29,62,6,15,7,12,14,23,18,13,6,2,17
3,2010,2010_3111,63,A,27,52,4,11,5,9,6,40,14,27,5,10,18
4,2010,2010_3119,74,H,30,74,7,20,7,11,14,33,18,11,5,3,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70778,2023,2023_3179,89,N,31,53,9,22,18,19,5,32,22,12,1,10,10
70779,2023,2023_3221,66,A,23,44,4,12,16,16,6,29,12,18,3,3,15
70780,2023,2023_3235,61,N,20,59,7,17,14,20,15,26,7,8,7,2,16
70781,2023,2023_3284,80,A,26,51,7,18,21,27,10,30,10,18,8,4,11


In [8]:

#create season of games
games=pd.concat([w_games,l_games],axis=0)
games=games.sort_values(['TeamID'])



In [9]:
games=pd.merge(games,conf.drop('Season',axis=1),left_on='TeamID',right_on='TeamID',how='left')
games=games.set_index('TeamID')


In [10]:
games.to_csv('Women_Games_by_team.csv')


In [12]:
baseline=games.groupby(['ConfAbbrev','Season']).mean()
games=games.reset_index()



/var/folders/jc/lq6pqgbn3dlfs9v5g5x4jc_80000gn/T/ipykernel_87250/2774296599.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  baseline=games.groupby(['ConfAbbrev','Season']).mean()


In [13]:
#ignore ranking columns
def plus_minus(data):
    #pick out the team, season and conference 
    team=data.copy()
    teamid=team['TeamID'].unique()[0]
    season=int(team['TeamID'].unique()[0].split('_')[0])
    con=team.ConfAbbrev.unique()[0]
    team=team.drop(['ConfAbbrev','TeamID','Loc','Season'],axis=1)
    #locate the team baseline
    conf_avg=baseline.loc[con,season]
    #sumtract and sume for plus minus
    pm=team-conf_avg
    pm=pm.sum(axis=0)
    pm['ConfAbbrev']=con
    return pm

In [15]:
pm=games.groupby('TeamID').progress_apply(plus_minus)
pm=pm.reset_index()

pm.head(5)

100%|██████████| 4880/4880 [00:05<00:00, 945.88it/s] 


,TeamID,Ast,Blk,DR,FGA,FGA3,FGM,FGM3,FTA,FTM,OR,PF,Score,Stl,TO,index,ConfAbbrev
0,2010_3102,-105.629213,-81.940075,-133.599251,-102.861423,-54.629213,-100.764045,-39.580524,-83.767790,-68.640449,-17.097378,-13.269663,-309.749064,-58.880150,28.074906,0.0,mwc
1,2010_3103,8.595568,-23.650970,-4.772853,-75.684211,-158.271468,-27.653740,-59.673130,2.792244,5.470914,8.825485,-10.177285,-109.509695,-38.141274,-12.858726,0.0,mac
2,2010_3104,-9.994536,-28.306011,34.765027,64.803279,-55.461749,-2.215847,-29.934426,-48.390710,-61.707650,1.480874,104.863388,-96.073770,-24.576503,83.204918,0.0,sec
3,2010_3105,-31.042403,49.604240,0.077739,-111.904594,-57.667845,1.604240,-11.498233,40.388693,58.674912,-18.349823,-21.759717,50.385159,-25.176678,92.590106,0.0,swac
4,2010_3106,17.176678,33.982332,40.342756,-57.897527,-70.717314,-54.017668,-22.424028,33.713781,21.021201,18.957597,61.332155,-109.438163,-48.597173,-43.625442,0.0,swac


In [16]:
pm.shape

(4880, 17)

In [17]:
seeds=pd.read_csv('/Users/sebastiancoombs/Downloads/march-machine-learning-mania-2023/WNCAATourneySeeds.csv')
seeds[seeds.TeamID==1246]


,Season,Seed,TeamID


In [18]:
seeds['Team']=seeds['TeamID']
seeds['TeamID']=seeds.Season.astype(str)+'_'+seeds.TeamID.astype(str)
# seeds=seeds.drop('Season',axis=1)
seeds.head(3)

,Season,Seed,TeamID,Team
0,1998,W01,1998_3330,3330
1,1998,W02,1998_3163,3163
2,1998,W03,1998_3112,3112


In [19]:
def get_team(data):
  return int(data.split('_')[-1])
  
def get_season(data):
  return int(data.split('_')[-0])

def fill_seed(data):
  data=data.fillna(np.random.randint(1,17))
  return data

In [20]:
pm['Team']=pm['TeamID'].apply(get_team)
pm['Season']=pm['TeamID'].apply(get_season)
pm.shape

(4880, 19)

In [21]:
len(list(seeds.Team.unique())),len(list(pm.Team.unique()))

(272, 363)

In [22]:
pm_seed=pd.merge(pm,seeds,left_on=['Team','Season'],right_on=['Team','Season'], how='left')

pm_seed['Seed']=pd.to_numeric(pm_seed['Seed'].str.replace("[a-zA-Z]","",regex=True),errors='coerce')
pm_seed['Seed']=pm_seed['Seed'].apply(lambda l: l if not np.isnan(l) else np.random.choice(range(1,17)))
pm_seed=pm_seed.drop('TeamID_y',axis=1)

pm_seed

,TeamID_x,Ast,Blk,DR,FGA,FGA3,FGM,FGM3,FTA,FTM,OR,PF,Score,Stl,TO,index,ConfAbbrev,Team,Season,Seed
0,2010_3102,-105.629213,-81.940075,-133.599251,-102.861423,-54.629213,-100.764045,-39.580524,-83.767790,-68.640449,-17.097378,-13.269663,-309.749064,-58.880150,28.074906,0.0,mwc,3102,2010,5.0
1,2010_3103,8.595568,-23.650970,-4.772853,-75.684211,-158.271468,-27.653740,-59.673130,2.792244,5.470914,8.825485,-10.177285,-109.509695,-38.141274,-12.858726,0.0,mac,3103,2010,4.0
2,2010_3104,-9.994536,-28.306011,34.765027,64.803279,-55.461749,-2.215847,-29.934426,-48.390710,-61.707650,1.480874,104.863388,-96.073770,-24.576503,83.204918,0.0,sec,3104,2010,9.0
3,2010_3105,-31.042403,49.604240,0.077739,-111.904594,-57.667845,1.604240,-11.498233,40.388693,58.674912,-18.349823,-21.759717,50.385159,-25.176678,92.590106,0.0,swac,3105,2010,16.0
4,2010_3106,17.176678,33.982332,40.342756,-57.897527,-70.717314,-54.017668,-22.424028,33.713781,21.021201,18.957597,61.332155,-109.438163,-48.597173,-43.625442,0.0,swac,3106,2010,11.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4875,2023_3473,-15.190813,-17.371025,-41.614841,-140.056537,101.236749,-62.272085,38.975265,-98.826855,-87.565371,-65.469965,-46.791519,-173.134276,-48.978799,62.212014,0.0,ovc,3473,2023,3.0
4876,2023_3474,-134.807107,-11.984772,-51.527919,25.294416,-60.593909,-76.802030,-39.812183,58.411168,35.664975,-5.157360,102.335025,-157.751269,5.076142,10.203046,0.0,a_sun,3474,2023,1.0
4877,2023_3475,13.321555,5.014134,74.480565,-17.978799,-36.713781,-11.522968,-10.865724,41.060071,55.212014,0.551237,16.296820,21.300353,-19.257951,81.420495,0.0,ovc,3475,2023,10.0
4878,2023_3476,63.665399,4.243346,19.193916,-18.840304,47.775665,-1.885932,20.650190,-46.878327,-19.437262,-24.117871,-61.045627,-2.558935,-91.365019,-40.787072,0.0,nec,3476,2023,15.0


In [23]:

# pm_seed.to_csv('NCAA_teams_plus_minus.csv')

In [24]:
pm_seed.shape

(4880, 20)

In [25]:
pm_seed[pm_seed.Seed.notna()]

,TeamID_x,Ast,Blk,DR,FGA,FGA3,FGM,FGM3,FTA,FTM,OR,PF,Score,Stl,TO,index,ConfAbbrev,Team,Season,Seed
0,2010_3102,-105.629213,-81.940075,-133.599251,-102.861423,-54.629213,-100.764045,-39.580524,-83.767790,-68.640449,-17.097378,-13.269663,-309.749064,-58.880150,28.074906,0.0,mwc,3102,2010,5.0
1,2010_3103,8.595568,-23.650970,-4.772853,-75.684211,-158.271468,-27.653740,-59.673130,2.792244,5.470914,8.825485,-10.177285,-109.509695,-38.141274,-12.858726,0.0,mac,3103,2010,4.0
2,2010_3104,-9.994536,-28.306011,34.765027,64.803279,-55.461749,-2.215847,-29.934426,-48.390710,-61.707650,1.480874,104.863388,-96.073770,-24.576503,83.204918,0.0,sec,3104,2010,9.0
3,2010_3105,-31.042403,49.604240,0.077739,-111.904594,-57.667845,1.604240,-11.498233,40.388693,58.674912,-18.349823,-21.759717,50.385159,-25.176678,92.590106,0.0,swac,3105,2010,16.0
4,2010_3106,17.176678,33.982332,40.342756,-57.897527,-70.717314,-54.017668,-22.424028,33.713781,21.021201,18.957597,61.332155,-109.438163,-48.597173,-43.625442,0.0,swac,3106,2010,11.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4875,2023_3473,-15.190813,-17.371025,-41.614841,-140.056537,101.236749,-62.272085,38.975265,-98.826855,-87.565371,-65.469965,-46.791519,-173.134276,-48.978799,62.212014,0.0,ovc,3473,2023,3.0
4876,2023_3474,-134.807107,-11.984772,-51.527919,25.294416,-60.593909,-76.802030,-39.812183,58.411168,35.664975,-5.157360,102.335025,-157.751269,5.076142,10.203046,0.0,a_sun,3474,2023,1.0
4877,2023_3475,13.321555,5.014134,74.480565,-17.978799,-36.713781,-11.522968,-10.865724,41.060071,55.212014,0.551237,16.296820,21.300353,-19.257951,81.420495,0.0,ovc,3475,2023,10.0
4878,2023_3476,63.665399,4.243346,19.193916,-18.840304,47.775665,-1.885932,20.650190,-46.878327,-19.437262,-24.117871,-61.045627,-2.558935,-91.365019,-40.787072,0.0,nec,3476,2023,15.0


In [26]:
pm_seed.shape

(4880, 20)

In [27]:
pm_seed[pm_seed.Seed.notna()]

,TeamID_x,Ast,Blk,DR,FGA,FGA3,FGM,FGM3,FTA,FTM,OR,PF,Score,Stl,TO,index,ConfAbbrev,Team,Season,Seed
0,2010_3102,-105.629213,-81.940075,-133.599251,-102.861423,-54.629213,-100.764045,-39.580524,-83.767790,-68.640449,-17.097378,-13.269663,-309.749064,-58.880150,28.074906,0.0,mwc,3102,2010,5.0
1,2010_3103,8.595568,-23.650970,-4.772853,-75.684211,-158.271468,-27.653740,-59.673130,2.792244,5.470914,8.825485,-10.177285,-109.509695,-38.141274,-12.858726,0.0,mac,3103,2010,4.0
2,2010_3104,-9.994536,-28.306011,34.765027,64.803279,-55.461749,-2.215847,-29.934426,-48.390710,-61.707650,1.480874,104.863388,-96.073770,-24.576503,83.204918,0.0,sec,3104,2010,9.0
3,2010_3105,-31.042403,49.604240,0.077739,-111.904594,-57.667845,1.604240,-11.498233,40.388693,58.674912,-18.349823,-21.759717,50.385159,-25.176678,92.590106,0.0,swac,3105,2010,16.0
4,2010_3106,17.176678,33.982332,40.342756,-57.897527,-70.717314,-54.017668,-22.424028,33.713781,21.021201,18.957597,61.332155,-109.438163,-48.597173,-43.625442,0.0,swac,3106,2010,11.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4875,2023_3473,-15.190813,-17.371025,-41.614841,-140.056537,101.236749,-62.272085,38.975265,-98.826855,-87.565371,-65.469965,-46.791519,-173.134276,-48.978799,62.212014,0.0,ovc,3473,2023,3.0
4876,2023_3474,-134.807107,-11.984772,-51.527919,25.294416,-60.593909,-76.802030,-39.812183,58.411168,35.664975,-5.157360,102.335025,-157.751269,5.076142,10.203046,0.0,a_sun,3474,2023,1.0
4877,2023_3475,13.321555,5.014134,74.480565,-17.978799,-36.713781,-11.522968,-10.865724,41.060071,55.212014,0.551237,16.296820,21.300353,-19.257951,81.420495,0.0,ovc,3475,2023,10.0
4878,2023_3476,63.665399,4.243346,19.193916,-18.840304,47.775665,-1.885932,20.650190,-46.878327,-19.437262,-24.117871,-61.045627,-2.558935,-91.365019,-40.787072,0.0,nec,3476,2023,15.0


In [28]:
teams=pd.read_csv('/Users/sebastiancoombs/Downloads/march-machine-learning-mania-2023/WTeams.csv')
teams[teams.TeamName=='Kentucky'].TeamID.unique()

array([3246])

In [29]:
data1=pd.merge(teams[['TeamID','TeamName']],pm_seed,left_on='TeamID',right_on='Team',how='left',)
data1['TeamID']=data1['TeamID_x']
data1=data1.drop(['TeamID_x'],axis=1)
first=list(data1.filter(like='Team').columns)
second=[c for c in data1.columns if c not in first]
order=first+second
data1=data1[order]
data1.columns

Index(['TeamID', 'TeamName', 'Team', 'Ast', 'Blk', 'DR', 'FGA', 'FGA3', 'FGM',
       'FGM3', 'FTA', 'FTM', 'OR', 'PF', 'Score', 'Stl', 'TO', 'index',
       'ConfAbbrev', 'Season', 'Seed'],
      dtype='object')

In [ ]:
data1.to_csv('Womens_pm_w_names.csv',index=False)